In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
chrome_exe_path=r"C:\software\chromex64\chrome.exe"
options = webdriver.ChromeOptions()

options.add_argument('--disable-gpu')
options.add_argument('--start-maximized')
# options.add_argument('--proxy-server=http://your_proxy_address:port')  # 设置代理地址和端口
if chrome_exe_path:
    options.binary_location = chrome_exe_path # 设置Chrome浏览器路径
# 使用webdriver-manager自动下载并启动chrome驱动
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service = service, options=options)



In [8]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
USERNAME = ""
PASSWORD = ""
driver.get("https://huggingface.co/login")
time.sleep(2)

# 输入用户名和密码
username = driver.find_element(By.NAME, "username")
password = driver.find_element(By.NAME, "password")

# 将 Hugging Face 账号的用户名和密码输入到对应的输入框
username.send_keys(USERNAME)  # 替换为实际的用户名
password.send_keys(PASSWORD)  # 替换为实际的密码
password.send_keys(Keys.RETURN)

# 等待登录完成
time.sleep(5)  # 根据网络情况调整等待时间

# 获取 cookies 并保存到文件中
cookies = driver.get_cookies()

In [9]:
import pickle  # 用于保存和加载 cookies
with open("huggingface_cookies.pkl", "wb") as file:
    pickle.dump(cookies, file)

In [15]:
driver.get("https://huggingface.co/blog/zh")
info_div = driver.find_element(By.XPATH,"/html/body/div/main/div/div[2]/div/div")


{'https://huggingface.co/blog/Suzana/interactive-tools': 'Interactive Tools for machine learning, deep learning, and math', 'https://huggingface.co/blog/mlabonne/abliteration': 'Uncensor any LLM with abliteration', 'https://huggingface.co/blog/sasha/energy-efficiency-bigger-better': "Bigger isn't always better: how to choose the most efficient model for context-specific tasks 🌱🧑🏼\u200d💻", 'https://huggingface.co/blog/Kseniase/mcp': '🦸🏻#14: What Is MCP, and Why Is Everyone – Suddenly!– Talking About It?', 'https://huggingface.co/blog/manu/conteb': '*Context Is Gold to Find the Gold Passage*: Evaluating and Training Contextual Document Embeddings', 'https://huggingface.co/blog/Nutanix/introducing-timesfm-for-time-series-forcasting': 'Building an Open Ecosystem for Time Series Forecasting: Introducing TimesFM in Hugging Face', 'https://huggingface.co/blog/codelion/system-prompt-learning': 'System Prompt Learning: Teaching LLMs to Learn Problem-Solving Strategies from Experience', 'https:/

In [ ]:
# 统计文章链接和标题
article_dict= {}
articles = info_div.find_elements(By.CSS_SELECTOR, "article")  # 或 By.XPATH, ".//article")
for article in articles:
    try:
        link = article.find_element(By.TAG_NAME, "a")
        url = link.get_attribute("href")
        title = link.find_element(By.TAG_NAME, "h4").text
        article_dict[url] = title
    except Exception as e:
        print(f"Error: {e}")
print(article_dict)

In [27]:
# 单个文章打开测试
first = next(iter(article_dict))
print(first)
driver.get("https://huggingface.co/blog/Suzana/interactive-tools")
# 定位正文容器
content_div = driver.find_element(By.XPATH, "/html/body/div/main/div/div[1]")

# 提取所有子元素（包括标题、段落、列表等）
elements = content_div.find_elements(By.XPATH, ".//*")

# 存储结构化文本
markdown_text = ""
for element in elements:
    tag_name = element.tag_name
    text = element.text.strip()

    if tag_name.startswith("h") and text:  # 处理标题（h1, h2, h3...）
        level = int(tag_name[1])  # 提取标题级别（如 h1 -> 1）
        markdown_text += f"{'#' * level} {text}\n\n"
    elif tag_name == "p" and text:  # 处理段落
        markdown_text += f"{text}\n\n"
    elif tag_name in ["ul", "ol"]:  # 处理列表
        items = element.find_elements(By.TAG_NAME, "li")
        for item in items:
            markdown_text += f"- {item.text}\n"
        markdown_text += "\n"

print(markdown_text)

https://huggingface.co/blog/Suzana/interactive-tools
# Interactive Tools for machine learning, deep learning, and math

- 
- 
- 
- 
- 
- 
- 

- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 
- 


























## Content

## Deep Learning

- Transformer Explainer
- exBERT
- BertViz
- CNN Explainer
- Play with GANs in the Browser
- ConvNet Playground
- Distill: Exploring Neural Networks with Activation Atlases
- A visual introduction to Machine Learning
- Interactive Deep Learning Playground
- Initializing neural networks
- Embedding Projector
- OpenAI Microscope

## Data

- Atlas Data Exploration

## Interpretability

- The Language Interpretability Tool
- What if
- Measuring diversity

## Math

- Sage Interactions
- Probability Distributions
- Bayesian Inference
- Seeing Theory: Probability and Stats
- Interactive Gaussian Process Visualization

# Deep Learning

## Transformer Explainer

Transformer Explainer is an interactive visualization tool design

In [ ]:
# 逐个打开文章获取文章内容
article_contents = {}
for url, title in article_dict.items():
    driver.get(url)
    time.sleep(2)  # 等待页面加载
    try:
        content_div = driver.find_element(By.CSS_SELECTOR, "div.markdown")
        content = content_div.text
        article_contents[url] = {
            'title': title,
            'content': content
        }
    except Exception as e:
        print(f"Error fetching content for {title}: {e}")